In [1]:
import pandas as pd
import pandas_profiling
import numpy as np
import re

In [2]:
df = pd.read_csv("invoice_nlp_data/gen_invoices_labels_100k.csv")

In [3]:
df.head()

,date,price,str_input,str_label
0,29.11.2022,0.99,"address : Perla Muniz 7000 Meadow Run Circle, ...",0000000000000000000000000000000000000000000000...
1,16.3.2028,0.21,Invoice\nAdventure Direct Internet\nDee Hacket...,0000000\n0000000000000000000000000\n0000000000...
2,28.4.2027,4.12,Invoice\nPower Provider\nJeremy Beers 1068 Sum...,0000000\n00000000000000\n000000000000000000000...
3,2026-23-7,4626.83,Electronic Interactive\nJeromy Mcghee 8226 Pel...,0000000000000000000000\n0000000000000000000000...
4,23.7.2020,0.37,address : Rolland Hutson 3023 Saint George Str...,0000000000000000000000000000000000000000000000...


In [4]:
# automatic EDA
# https://nbviewer.jupyter.org/github/JosPolfliet/pandas-profiling/blob/master/examples/meteorites.ipynb
# pandas_profiling.ProfileReport(df)

In [19]:
inpu = []
targ = []
i = 0

for i in range(len(df['str_input'])):
    inp = df['str_input'][i].split("\\n")
    tar = df['str_label'][i].split("\\n")

    inp = [x for x in inp if len(x)>0]
    tar = [x for x in tar if len(x)>0]

    inpu.append(inp)
    targ.append(tar)

# for safety
inpu = [x for x in inpu if len(x)>0]
targ = [x for x in targ if len(x)>0]

inpu = np.concatenate(inpu)
targ = np.concatenate(targ)

In [32]:
inpu[103], targ[103], inpu[-19], targ[-19]

('price EUR 0.24', '00000000001111', 'price EUR 16.15', '000000000011111')

# Tests

In [96]:
chars = sorted(list(set(' '.join(df['str_input'][:10000]) + ' '.join(df['str_label'][:10000]))))
mapping = dict((c, i) for i, c in enumerate(chars))

In [97]:
len(mapping)

75

In [179]:
# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 75


In [182]:
sequences = list()
for line in inpu:
    # integer encode line
    encoded_seq = [mapping[char] for char in line]
    # store
    sequences.append(encoded_seq)

In [188]:
np.array(sequences)

array([list([48, 51, 51, 65, 52, 66, 66, 0, 18, 0, 35, 52, 65, 59, 48, 0, 32, 68, 61, 56, 73, 0, 15, 8, 8, 8, 0, 32, 52, 48, 51, 62, 70, 0, 37, 68, 61, 0, 22, 56, 65, 50, 59, 52, 4, 0, 9, 8, 13, 16, 9, 0, 28, 65, 48]),
       list([10, 17, 6, 9, 9, 6, 10, 8, 10, 10]),
       list([23, 52, 66, 56, 54, 61, 0, 22, 62, 61, 66, 67, 65, 68, 50, 67, 56, 62, 61, 0, 33, 52, 67, 70, 62, 65, 58]),
       ...,
       list([23, 62, 59, 62, 65, 52, 0, 52, 71, 0, 60, 56, 61, 56, 60, 0, 52, 67, 48, 50, 50, 68, 60, 66, 48, 61, 0, 63, 65, 48, 52, 66, 52, 61, 67, 6]),
       list([24, 67, 48, 50, 50, 68, 60, 66, 48, 61, 0, 64, 68, 56, 0, 52, 68, 60, 0, 49, 59, 48, 61, 51, 56, 67, 0, 48, 51, 0, 40, 67, 70, 56, 66, 56, 0, 62, 51, 56, 62, 6]),
       list([])], dtype=object)

In [186]:
# sequences = array(sequences)
type(np.array(sequences)), type(targ)

(numpy.ndarray, numpy.ndarray)

In [183]:
print(len(sequences), len(targ))

1799964 1799964


In [ ]:
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)

# LM-LSTM-CRF

In [25]:
# https://github.com/LiyuanLucasLiu/LM-LSTM-CRF

In [26]:
# import sys
# sys.path.append("LT_LSTM_CRF")
# import model.utils as utils

In [27]:
# # converting format
# train_features, train_labels, f_map, l_map, c_map = utils.generate_corpus_char(inpu, if_shrink_c_feature=True, c_thresholds=args.mini_count, if_shrink_w_feature=False)


# Simple Bi-LSTM CRF

In [42]:
# https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html

# Viterbi algorithm - calculates maximum probability path for a hidden markov model

In [44]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split

torch.manual_seed(1)

In [34]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [45]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

char_to_ix = {}
for sentence in inpu:
    for char in sentence:
        if char not in char_to_ix:
            char_to_ix[char] = len(char_to_ix)

tag_to_ix = {}
for sentence in targ:
    for char in sentence:
        if char not in tag_to_ix:
            tag_to_ix[char] = len(tag_to_ix)

tag_to_ix[START_TAG] = len(tag_to_ix)
tag_to_ix[STOP_TAG] = len(tag_to_ix)

all_data = np.transpose(np.stack([np.array(inpu), np.array(targ)]))

In [46]:
training_data, test_data = train_test_split(all_data, test_size=0.2)

In [52]:
model = BiLSTM_CRF(len(char_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

In [53]:
# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], char_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print(model(precheck_sent))

(tensor(14.8086), [2, 2, 2, 2, 2, 2, 2, 2, 2])


In [ ]:
# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(1):
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, char_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)
        
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()
    
# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], char_to_ix)
    print(training_data[0][0])
    print(model(precheck_sent))
# We got it!

In [12]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [56]:
def init_hidden():
    return (torch.randn(2, 1, hidden_dim // 2),
            torch.randn(2, 1, hidden_dim // 2))

def _forward_alg(feats):
    # Do the forward algorithm to compute the partition function
    init_alphas = torch.full((1, tagset_size), -10000.)
    # START_TAG has all of the score.
    init_alphas[0][tag_to_ix[START_TAG]] = 0.

    # Wrap in a variable so that we will get automatic backprop
    forward_var = init_alphas

    # Iterate through the sentence
    for feat in feats:
        alphas_t = []  # The forward tensors at this timestep
        for next_tag in range(tagset_size):
            # broadcast the emission score: it is the same regardless of
            # the previous tag
            emit_score = feat[next_tag].view(
                1, -1).expand(1, tagset_size)
            # the ith entry of trans_score is the score of transitioning to
            # next_tag from i
            trans_score = transitions[next_tag].view(1, -1)
            # The ith entry of next_tag_var is the value for the
            # edge (i -> next_tag) before we do log-sum-exp
            next_tag_var = forward_var + trans_score + emit_score
            # The forward variable for this tag is log-sum-exp of all the
            # scores.
            alphas_t.append(log_sum_exp(next_tag_var).view(1))
        forward_var = torch.cat(alphas_t).view(1, -1)
    terminal_var = forward_var + transitions[tag_to_ix[STOP_TAG]]
    alpha = log_sum_exp(terminal_var)
    return alpha

def _get_lstm_features(sentence):
    hidden = init_hidden()
    embeds = word_embeds(sentence).view(len(sentence), 1, -1)
    lstm_out, hidden = lstm(embeds, hidden)
    lstm_out = lstm_out.view(len(sentence), hidden_dim)
    lstm_feats = hidden2tag(lstm_out)
    return lstm_feats

def _score_sentence(feats, tags):
    # Gives the score of a provided tag sequence
    score = torch.zeros(1)
    tags = torch.cat([torch.tensor([tag_to_ix[START_TAG]], dtype=torch.long), tags])
    for i, feat in enumerate(feats):
        score = score + \
            transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
    score = score + transitions[tag_to_ix[STOP_TAG], tags[-1]]
    return score

def _viterbi_decode(feats):
    backpointers = []

    # Initialize the viterbi variables in log space
    init_vvars = torch.full((1, tagset_size), -10000.)
    init_vvars[0][tag_to_ix[START_TAG]] = 0

    # forward_var at step i holds the viterbi variables for step i-1
    forward_var = init_vvars
    for feat in feats:
        bptrs_t = []  # holds the backpointers for this step
        viterbivars_t = []  # holds the viterbi variables for this step

        for next_tag in range(tagset_size):
            # next_tag_var[i] holds the viterbi variable for tag i at the
            # previous step, plus the score of transitioning
            # from tag i to next_tag.
            # We don't include the emission scores here because the max
            # does not depend on them (we add them in below)
            next_tag_var = forward_var + transitions[next_tag]
            best_tag_id = argmax(next_tag_var)
            bptrs_t.append(best_tag_id)
            viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
        # Now add in the emission scores, and assign forward_var to the set
        # of viterbi variables we just computed
        forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
        backpointers.append(bptrs_t)

    # Transition to STOP_TAG
    terminal_var = forward_var + transitions[tag_to_ix[STOP_TAG]]
    best_tag_id = argmax(terminal_var)
    path_score = terminal_var[0][best_tag_id]

    # Follow the back pointers to decode the best path.
    best_path = [best_tag_id]
    for bptrs_t in reversed(backpointers):
        best_tag_id = bptrs_t[best_tag_id]
        best_path.append(best_tag_id)
    # Pop off the start tag (we dont want to return that to the caller)
    start = best_path.pop()
    assert start == tag_to_ix[START_TAG]  # Sanity check
    best_path.reverse()
    return path_score, best_path

def neg_log_likelihood(sentence, tags):
    feats = _get_lstm_features(sentence)
    forward_score = _forward_alg(feats)
    gold_score = _score_sentence(feats, tags)
    return forward_score - gold_score

def forward(sentence):  # dont confuse this with _forward_alg above.
    # Get the emission scores from the BiLSTM
    lstm_feats = _get_lstm_features(sentence)

    # Find the best path, given the features.
    score, tag_seq = _viterbi_decode(lstm_feats)
    return score, tag_seq

In [57]:
embedding_dim = EMBEDDING_DIM
hidden_dim = HIDDEN_DIM
vocab_size = len(char_to_ix)
tag_to_ix = tag_to_ix
tagset_size = len(tag_to_ix)

word_embeds = nn.Embedding(vocab_size, embedding_dim)
lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                    num_layers=1, bidirectional=True)

# Maps the output of the LSTM into tag space.
hidden2tag = nn.Linear(hidden_dim, tagset_size)

# Matrix of transition parameters.  Entry i,j is the score of
# transitioning *to* i *from* j.
transitions = nn.Parameter(
    torch.randn(tagset_size, tagset_size))

# These two statements enforce the constraint that we never transfer
# to the start tag and we never transfer from the stop tag
transitions.data[tag_to_ix[START_TAG], :] = -10000
transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

hidden = init_hidden()

In [58]:
# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(1):
    for sentence, tags in training_data[:5]:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, char_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)
        
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()
    
# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], char_to_ix)
    print(training_data[0][0])
    print(model(precheck_sent))
# We got it!

4.10.2028
(tensor(17.5286), [0, 0, 0, 0, 0, 0, 0, 0, 0])


In [59]:
model.zero_grad()

In [66]:
sentence, tags = training_data[0]

In [67]:
sentence_in = prepare_sequence(sentence, char_to_ix); sentence_in

tensor([49, 28, 22, 15, 28, 26, 15, 26, 24])

In [68]:
targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long); targets

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1])

In [83]:
import time

In [105]:
from timeit import Timer
t = Timer("_forward_alg(feats)")
print(t.timeit())

NameError: name '_forward_alg' is not defined

In [88]:
(sentence, tags) = (sentence_in, targets)
feats = _get_lstm_features(sentence)
feats

tensor([[-0.2456, -0.0374,  0.5114, -0.1389,  0.2680],
        [-0.1953, -0.0759,  0.5497, -0.1208,  0.2409],
        [-0.1458, -0.1258,  0.5769, -0.0876,  0.2555],
        [-0.0669, -0.1876,  0.5629, -0.1414,  0.1615],
        [-0.1339, -0.1388,  0.4942, -0.1299,  0.2259],
        [-0.0853, -0.1675,  0.6249, -0.0337,  0.2307],
        [-0.0687, -0.1885,  0.5714, -0.1299,  0.1768],
        [-0.1213, -0.1415,  0.5773, -0.0427,  0.2559],
        [-0.1362, -0.1312,  0.5842, -0.0919,  0.2404]],
       grad_fn=<AddmmBackward>)

In [97]:
# def _forward_alg_new(feats):
# Do the forward algorithm to compute the partition function
init_alphas = torch.full((1, tagset_size), -10000.)
# START_TAG has all of the score.
init_alphas[0][tag_to_ix[START_TAG]] = 0.

# Wrap in a variable so that we will get automatic backprop
forward_var = init_alphas

# Iterate through the sentence
for feat in feats:
    alphas_t = []  # The forward tensors at this timestep
    for next_tag in range(tagset_size):
        # broadcast the emission score: it is the same regardless of
        # the previous tag
        emit_score = feat[next_tag].view(
            1, -1).expand(1, tagset_size)
        # the ith entry of trans_score is the score of transitioning to
        # next_tag from i
        trans_score = transitions[next_tag].view(1, -1)
        # The ith entry of next_tag_var is the value for the
        # edge (i -> next_tag) before we do log-sum-exp
        next_tag_var = forward_var + trans_score + emit_score
        # The forward variable for this tag is log-sum-exp of all the
        # scores.
        alphas_t.append(log_sum_exp(next_tag_var).view(1))
        
        print(forward_var, "\n", next_tag, "\n", emit_score, "\n", trans_score, "\n\n\n")
    forward_var = torch.cat(alphas_t).view(1, -1)
terminal_var = forward_var + transitions[tag_to_ix[STOP_TAG]]
alpha = log_sum_exp(terminal_var)
# return alpha


tensor([[-10000., -10000., -10000.,      0., -10000.]]) 
 0 
 tensor([[-0.2456, -0.2456, -0.2456, -0.2456, -0.2456]],
       grad_fn=<ExpandBackward>) 
 tensor([[-1.3550e+00, -2.1336e-02, -8.9116e-01, -7.5454e-02, -1.0000e+04]],
       grad_fn=<ViewBackward>) 



tensor([[-10000., -10000., -10000.,      0., -10000.]]) 
 1 
 tensor([[-0.0374, -0.0374, -0.0374, -0.0374, -0.0374]],
       grad_fn=<ExpandBackward>) 
 tensor([[ 1.3932e-02,  4.4707e-01, -7.8531e-01, -1.8161e-01, -1.0000e+04]],
       grad_fn=<ViewBackward>) 



tensor([[-10000., -10000., -10000.,      0., -10000.]]) 
 2 
 tensor([[0.5114, 0.5114, 0.5114, 0.5114, 0.5114]], grad_fn=<ExpandBackward>) 
 tensor([[ 1.6299e+00,  9.3079e-01,  1.1976e+00, -1.8072e-01, -1.0000e+04]],
       grad_fn=<ViewBackward>) 



tensor([[-10000., -10000., -10000.,      0., -10000.]]) 
 3 
 tensor([[-0.1389, -0.1389, -0.1389, -0.1389, -0.1389]],
       grad_fn=<ExpandBackward>) 
 tensor([[-10000., -10000., -10000., -10000., -10000.]], grad_fn=<Vi

In [104]:
feat[0:5]

tensor([[-0.1362, -0.1312,  0.5842, -0.0919,  0.2404]],
       grad_fn=<ExpandBackward>)

In [98]:
feat

tensor([-0.1362, -0.1312,  0.5842, -0.0919,  0.2404], grad_fn=<SelectBackward>)

In [92]:
alphas_t

[tensor([13.6405], grad_fn=<ViewBackward>),
 tensor([13.8936], grad_fn=<ViewBackward>),
 tensor([16.3885], grad_fn=<ViewBackward>),
 tensor([-9985.3955], grad_fn=<ViewBackward>),
 tensor([14.2884], grad_fn=<ViewBackward>)]

In [91]:
forward_var

tensor([[   13.6405,    13.8936,    16.3885, -9985.3955,    14.2884]],
       grad_fn=<ViewBackward>)

In [85]:
(sentence, tags) = (sentence_in, targets)
feats = _get_lstm_features(sentence)

start = time.time()
for i in range(1000):
    forward_score = _forward_alg(feats)
end = time.time()
print(end - start)

gold_score = _score_sentence(feats, tags)
loss = forward_score - gold_score

3.9899041652679443


In [75]:
forward_score

tensor(15.2857, grad_fn=<AddBackward0>)